In [4]:
import subprocess
def run_cmd(cmd):
    ret = subprocess.run(cmd.split())
    if ret.returncode != 0:
        print("error: ", ret.returncode)

In [5]:
# Build the shared object and load it as a module
run_cmd("make -C ../LinuxPrototypes/python/ clean")
run_cmd("make -C ../LinuxPrototypes/python/ my_shared_object.so")

make: Entering directory '/home/sym/Symbi-OS/LinuxPrototypes/python'
rm -f *.o *.so
make: Leaving directory '/home/sym/Symbi-OS/LinuxPrototypes/python'
make: Entering directory '/home/sym/Symbi-OS/LinuxPrototypes/python'
as -o my_assembly_function.o my_assembly_function.S
gcc -shared -o my_shared_object.so my_assembly_function.o
make: Leaving directory '/home/sym/Symbi-OS/LinuxPrototypes/python'


In [2]:
import ctypes

# On x86, e.g. Intel processors, Control Register 3 (cr3) is a protected system register.
# It must neither be read nor written by user code, or the system can be compromised.

# This module contains an evil read cr3 instruction! 👻
evil_mod = ctypes.cdll.LoadLibrary("../LinuxPrototypes/python/my_shared_object.so")

# The Symbiote library: used here to exercise the new syscall we added to the Linux kernel.
# We use this to enter the privileged "supervisor mode", allowing us access a superset of 
# ISA (machine code) instructions, for example, reading cr3, e.g. "mov cr3, %rax".
symlib_mod = ctypes.cdll.LoadLibrary("../Symlib/dynam_build/libSym.so")

In [3]:
# UNIT TEST (intended to PASS). 
# On PASS you will see "CR3 = 0x..."

# Elevate permission: use elevate syscall to enter supervisor mode. 
symlib_mod.sym_elevate()

# Read cr3! Attempt a "mov cr3, %rax" instruction.
cr3 = evil_mod.getcr3()

# Lower permission: return back to user mode.
symlib_mod.sym_lower()

print("CR3 = 0x{:x}".format(cr3))

CR3 = 0x664ee004


In [4]:
# UNIT TEST (INTENDED TO FAIL): 
# On FAIL you will see "Canceled future ..." and the python kernel crash.

# Standard user mode process, the processor catches the attmepted "mov cr3, %rax"
# instruction with a general protection fault, and kills the offending process, 
# the python interpreter, or "Kernel". The print will never come!
cr3 = evil_mod.getcr3()
print("CR3 = 0x{:x}".format(cr3))

: 

: 